## 262. Trips and Users (Hard)

In [ ]:
Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03".

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Return the result table in any order. Round Cancellation Rate to two decimal points.

The query result format is in the following example:
    
    Table: Trips

+-------------+----------+  
| Column Name | Type     |  
+-------------+----------+  
| Id          | int      |  
| Client_Id   | int      |  
| Driver_Id   | int      |  
| City_Id     | int      |  
| Status      | enum     |  
| Request_at  | date     |       
+-------------+----------+  

Id is the primary key for this table.
The table holds all taxi trips. Each trip has a unique Id, while Client_Id and Driver_Id are foreign keys to the Users_Id at the Users table.
Status is an ENUM type of (‘completed’, ‘cancelled_by_driver’, ‘cancelled_by_client’).

    Table: Users
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| Users_Id    | int      |
| Banned      | enum     |
| Role        | enum     |
+-------------+----------+

Users_Id is the primary key for this table.
The table holds all users. Each user has a unique Users_Id, and Role is an ENUM type of (‘client’, ‘driver’, ‘partner’).
Status is an ENUM type of (‘Yes’, ‘No’).


In [ ]:
WITH CTE AS (SELECT * FROM trips
             WHERE client_id IN (SELECT users_id
                                 FROM users 
                                 WHERE banned = 'No')
             AND driver_id IN (SELECT users_id
                               FROM users 
                               WHERE banned = 'No')
             AND request_at BETWEEN '2013-10-01' and '2013-10-03')
 
 
 SELECT request_at AS 'Day', 
        ROUND(CAST(SUM(CASE 
                    WHEN status = 'completed'
                    THEN 0 
                    ELSE 1 
                  END) AS DECIMAL(18,2)) / 
              COUNT(*), 2)
             AS 'Cancellation Rate'
 FROM CTE
 GROUP BY request_at;

## 601. Human Traffic of Stadium (Hard)

In [ ]:
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| visit_date    | date    |
| people        | int     |
+---------------+---------+
visit_date is the primary key for this table.
Each row of this table contains the visit date and visit id to the stadium with the number of people during the visit.
No two rows will have the same visit_date, and as the id increases, the dates increase as well.
 

Write an SQL query to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The query result format is in the following example.

 

Stadium table:
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+

Result table:
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
The four rows with ids 5, 6, 7, and 8 have consecutive ids and each of them has >= 100 people attended. Note that row 8 was included even though the visit_date was not the next day after row 7.
The rows with ids 2 and 3 are not included because we need at least three consecutive ids.

In [ ]:
# Write your T-SQL query statement below

WITH CTE1 AS (
    SELECT 
        (id - RANK() OVER (ORDER BY id)) as [pointer], -- same number for consecutive ids
        id,
        visit_date,
        people
    FROM Stadium
    WHERE people > 99
),
    CTE2 AS (
    SELECT 
        COUNT(id) OVER (PARTITION BY [pointer]) as [three_or_more], -- count number of consecutive ids
        id,
        visit_date,
        people
    FROM CTE1
)

SELECT
    id, visit_date, people
FROM CTE2 WHERE [three_or_more] > 2

## 185. Department Top Three Salaries (Hard)

In [ ]:
The Employee table holds all employees. Every employee has an Id, and there is also a column for the department Id.

+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |
+----+-------+--------+--------------+
The Department table holds all departments of the company.

+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
Write a SQL query to find employees who earn the top three salaries in each of the department. For the above tables, your SQL query should return the following rows (order of rows does not matter).

+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Randy    | 85000  |
| IT         | Joe      | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+
Explanation:

In IT department, Max earns the highest salary, both Randy and Joe earn the second highest salary, and Will earns the third highest salary. There are only two employees in the Sales department, Henry earns the highest salary while Sam earns the second highest salary.



In [ ]:
# Write your T-SQL query statement below 

WITH CTE AS (
    SELECT 
        DENSE_RANK() OVER (PARTITION BY DepartmentId ORDER BY Salary DESC) AS [dense_rank]
        ,Id
        ,Name
        ,Salary
        ,DepartmentId
    FROM Employee
)

SELECT 
    d.Name AS [Department]
    ,CTE.Name AS [Employee]
    ,Salary
FROM CTE 
JOIN Department d on d.Id = CTE.DepartmentId
WHERE [dense_rank] < 4

## 626. Exchange Seats (medium)

In [ ]:
Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids.

The column id is continuous increment.

Mary wants to change seats for the adjacent students.

Can you write a SQL query to output the result for Mary?

 

+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Abbot   |
|    2    | Doris   |
|    3    | Emerson |
|    4    | Green   |
|    5    | Jeames  |
+---------+---------+
For the sample input, the output is:

+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Doris   |
|    2    | Abbot   |
|    3    | Green   |
|    4    | Emerson |
|    5    | Jeames  |
+---------+---------+
Note:

If the number of students is odd, there is no need to change the last one's seat.

In [1]:
from math import floor, ceil

In [25]:
for i in range(1,16):
    print(i, ceil(i/2))

1 1
2 1
3 2
4 2
5 3
6 3
7 4
8 4
9 5
10 5
11 6
12 6
13 7
14 7
15 8


In [24]:
for i in range(1,16):
    print("sum: " + str(floor(i/2) + (i % 2)))

sum: 1
sum: 1
sum: 2
sum: 2
sum: 3
sum: 3
sum: 4
sum: 4
sum: 5
sum: 5
sum: 6
sum: 6
sum: 7
sum: 7
sum: 8


In [2]:
ceil(0.5) # t-sql returns 1 also

1

In [3]:
ceil(1/2) # t-sql returns 0 

1

In [ ]:
# Write your T-SQL query statement below 

"""
SELECT ROW_NUMBER() OVER (ORDER BY FLOOR(id / 2) + (id % 2), id % 2) AS id, student
FROM seat
"""

## 177. Nth Highest Salary

In [ ]:
Write a SQL query to get the nth highest salary from the Employee table.

+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
For example, given the above Employee table, the nth highest salary where n = 2 is 200. If there is no nth highest salary, then the query should return null.

+------------------------+
| getNthHighestSalary(2) |
+------------------------+
| 200                    |
+------------------------+

In [ ]:
CREATE FUNCTION getNthHighestSalary(@N INT) RETURNS INT AS
BEGIN
    RETURN (
        /* Write your T-SQL query statement below. */
        SELECT max(Salary) FROM (
        SELECT 
        Salary,
        dense_rank() over (order by Salary desc) as Nth
        FROM Employee ) t
        WHERE Nth = @N
    );
END